In [ ]:
pip install torch torchvision torchaudio yolov5 matplotlib shutil

In [ ]:
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from yolov5 import YOLOv5

In [ ]:
pretrained_file_path = '../yolov5m.pt'
model = YOLOv5(pretrained_file_path)

if os.path.exists(pretrained_file_path):
    model = YOLOv5(pretrained_file_path)
    print("Model loaded successfully.")
else:
    raise FileNotFoundError(f"File {pretrained_file_path} not found.")

In [ ]:
# Define source and destination directories
src_dir = '../data/cam1'
dst_dir = '../data/cam1_rgb'

# Create destination directory if it doesn't exist
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Function to delete a directory if it exists
def delete_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)

# Function to recreate a directory
def recreate_directory(directory):
    os.makedirs(directory)

# Function to process images
def process_images(src_dir, dst_dir):
    for filename in os.listdir(src_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Read the image
            image_path = os.path.join(src_dir, filename)
            image = cv2.imread(image_path)
            
            # Ensure the image is loaded correctly
            if image is not None:
                # Get the right half of the image (640x928)
                height, width = image.shape[:2]
                right_half = image[:, width//2:]

                # Save the right half to the destination directory
                dst_image_path = os.path.join(dst_dir, filename)
                cv2.imwrite(dst_image_path, right_half)
            else:
                print(f"Failed to load image: {image_path}")

# Process the images
process_images(src_dir, dst_dir)

print(f"Processing complete. The right half of the images have been saved to {dst_dir}")

In [ ]:
# Define the directory containing the JSON files
annotation_dir = '../data/cam1_labeled'

# List all files in the annotation directory
files = os.listdir(annotation_dir)

# Loop through the files and rename JSON files
for filename in files:
    if filename.endswith('_json.json'):
        new_filename = filename.replace('_json', '')
        old_filepath = os.path.join(annotation_dir, filename)
        new_filepath = os.path.join(annotation_dir, new_filename)
        os.rename(old_filepath, new_filepath)
        print(f'Renamed: {filename} to {new_filename}')

In [ ]:
import json
import os
from PIL import Image, ImageDraw

def load_annotations(filepath):
    with open(filepath, 'r') as file:
        annotations = json.load(file)
    return annotations

def segment_image(image_path, annotations):
    try:
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        
        image_filename = os.path.basename(image_path)
        if image_filename in annotations:
            regions = annotations[image_filename]['regions']
            for region in regions:
                shape_attributes = region['shape_attributes']
                x = shape_attributes['x']
                y = shape_attributes['y']
                width = shape_attributes['width']
                height = shape_attributes['height']
                draw.rectangle([x, y, x + width, y + height], outline='red')
        else:
            raise ValueError(f"Annotations for image {image_filename} not found")
        
        return image
    except Exception as e:
        print(f"Error in segment_image: {e}")
        raise

# Example usage
annotation_path = '../data/cam1_labeled/date_1-5-2024_10.0.11_1_labeled.json'
image_path = '../data/cam1_labeled/date_1-5-2024_10.0.11_1_labeled.png'

try:
    annotations = load_annotations(annotation_path)
    # print('Loaded annotations:', annotations)
    
    segmented_image = segment_image(image_path, annotations)
    if segmented_image:
        segmented_image.show()
        
        # Save the segmented image
        output_path = os.path.splitext(image_path)[0] + "_segmented.png"
        segmented_image.save(output_path)
        print(f"Segmented image saved to {output_path}")
    else:
        print("Error: Segmented image is None.")

except FileNotFoundError as e:
    print(f"Error: File not found - {e}")
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Example usage
annotation_path = '../data/cam1_labeled/date_1-5-2024_10.0.11_1_labeled.json'
image_path = '../data/cam1_labeled/date_1-5-2024_10.0.11_1_labeled.png'

try:
    annotations = load_annotations(annotation_path)
    print('Loaded annotations:', annotations) # prints the annotations
    
    segmented_image = segment_image(image_path, annotations) # doesn't work after this line
    segmented_image.show()
    
    # Save the segmented image
    output_path = os.path.splitext(image_path)[0] + "_segmented.png"
    segmented_image.save(output_path)
    print(f"Segmented image saved to {output_path}")

except FileNotFoundError as e:
    print(f"Error: File not found - {e}")
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: {e}")

### After Use, delete the directories

In [ ]:
delete_directory('../data/cam1_rgb')

# delete_directory('../data/cam1_labeled')